<a href="https://colab.research.google.com/github/Marcin19721205/IntroductionToBigDataWSB/blob/main/Zadanie_Nr5_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analiza koszykowa
Przeanalizuj przykład zamieszczony w pliku analiza_koszykowa_01a_s.r . Następnie przeprowadź w
języku R analizę koszykową za pomocą algorytmu Apriori dla transakcji zakupowych wczytanych z
podanego niżej pliku.  

zadanie w Python

Wczytaj transakcje z pliku ItemList.csv . Uwaga: Plik z transakcjami nie jest klasycznym
plikiem w formacie CSV i należy go wczytać następująco

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#wczytaj dane
transactions_list = []
with open('sample_data/ItemList.csv', 'r') as file:
    for line in file:
        # Split by comma, strip whitespace from each item, and filter out empty strings
        items = [item.strip() for item in line.strip().split(',') if item.strip()]
        if items: # Only add non-empty transactions
            transactions_list.append(items)

# DataFrame
df = pd.DataFrame({'items': transactions_list})
#
#podgląd
print("Pierwsze 5 transakcji:")
for i, trans in enumerate(transactions_list[:5]):
    print(f"Transakcja {i+1}: {trans}")

print("\nInformacje o danych (DataFrame z listą przedmiotów):")
print(df.head())
print(df.info())

Pierwsze 5 transakcji:
Transakcja 1: ['itemList']
Transakcja 2: ['1', 'sausage', 'whole milk', 'semi-finished bread', 'yogurt']
Transakcja 3: ['2', 'whole milk', 'pastry', 'salty snack']
Transakcja 4: ['3', 'canned beer', 'misc. beverages']
Transakcja 5: ['4', 'sausage', 'hygiene articles']

Informacje o danych (DataFrame z listą przedmiotów):
                                               items
0                                         [itemList]
1  [1, sausage, whole milk, semi-finished bread, ...
2               [2, whole milk, pastry, salty snack]
3                  [3, canned beer, misc. beverages]
4                     [4, sausage, hygiene articles]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14964 entries, 0 to 14963
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   items   14964 non-null  object
dtypes: object(1)
memory usage: 117.0+ KB
None


#punkt b

Za pomocą algorytmu Apriori wylicz reguły wynikowe dla minimalnego wsparcia = 0.01 oraz
minimalnej pewności = 0.01 . Wylicz tylko reguły długości 2 (argument minlen=2) tzn.
zawierające przynajmniej 2 produkty.

In [25]:
#Analiza asocjacji (Apriori)
import re, warnings
warnings.filterwarnings("ignore", message=r".*datetime\.utcnow\(\) is deprecated.*")
#
# Wczytanie pliku "basket" (1 wiersz = 1 koszyk; 1. kolumna = ID) ----
def read_basket_file(path: Path):
    tx = []
    with open(path, 'r', encoding='utf-8') as f:
        for ln, raw in enumerate(f):
            line = raw.strip()
            if not line:
                continue
            items = [x.strip() for x in line.split(',') if x.strip()]

            # pomiń nagłówek typu 'itemList'
            if ln == 0 and len(items) == 1 and items[0].lower() in {"itemlist","items"}:
                continue
            # usuń ID transakcji (1. token liczbowy)
            if items and re.fullmatch(r"\d+", items[0]):
                items = items[1:]

            # deduplikacja w koszyku (jak rm.duplicates=TRUE)
            seen = set()
            deduped = [x for x in items if not (x in seen or seen.add(x))]
            if deduped:
                tx.append(deduped)
    return tx

#Ścieżka do pliku
basket_path = Path("sample_data/ItemList.csv")   # podmień, jeśli masz inną lokalizację
transactions = read_basket_file(basket_path)

print(f"Transakcji: {len(transactions)} | Przykładowe: {transactions[:2]}")

#One-hot
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_tx = pd.DataFrame(te_ary, columns=te.columns_)   # wiersz=koszyk, kolumna=produkt, True/False

#Zbiory częste (Apriori; sup ≥ 0.01
freq = apriori(df_tx, min_support=0.01, use_colnames=True)
freq = freq.sort_values(['support','itemsets'], ascending=[False, True]).reset_index(drop=True)
print("\nTOP zbiory częste (support):")
print(freq.head(10))

#Reguły A→B (conf ≥ 0.01), + minlen=2 (co najmniej 1 w LHS i 1 w RHS
rules = association_rules(freq, metric="confidence", min_threshold=0.01)
rules = rules[(rules['antecedents'].apply(len) >= 1) & (rules['consequents'].apply(len) >= 1)]

# czytelne kolumny
rules_disp = rules.copy()
rules_disp['antecedents'] = rules_disp['antecedents'].apply(lambda s: ', '.join(sorted(list(s))))
rules_disp['consequents'] = rules_disp['consequents'].apply(lambda s: ', '.join(sorted(list(s))))
rules_disp = rules_disp[['antecedents','consequents','support','confidence','lift']]

print(f"\nReguł (minlen=2): {len(rules_disp)}")
print("\nTOP wg support:")
print(rules_disp.sort_values('support', ascending=False).head(20))

print("\nTOP wg confidence:")
print(rules_disp.sort_values('confidence', ascending=False).head(20))

print("\nTOP wg lift:")
print(rules_disp.sort_values('lift', ascending=False).head(20))

#
#

fig = px.scatter(
    rules_disp,
    x='support', y='confidence',
    color='lift', size='lift',
    hover_data=['antecedents','consequents'],
    title='Reguły: support vs confidence (kolor = lift)'
)
fig.show()

# filtr reguł „sensownych”
rules_good = rules_disp.query('lift > 1')

fig2 = px.scatter(
    rules_good, x='support', y='confidence',
    color='lift', size='lift',
    hover_data=['antecedents','consequents'],
    title='Reguły (lift > 1): support vs confidence'
)
# zapis do PNG/HTML (w Colab wymaga kaleido)
# pip install -q kaleido
# fig2.write_image('rules_scatter.png', scale=2)
fig2.write_html('rules_scatter.html')  # zawsze działa



Transakcji: 14963 | Przykładowe: [['sausage', 'whole milk', 'semi-finished bread', 'yogurt'], ['whole milk', 'pastry', 'salty snack']]

TOP zbiory częste (support):
    support            itemsets
0  0.157923        (whole milk)
1  0.122101  (other vegetables)
2  0.110005        (rolls/buns)
3  0.097106              (soda)
4  0.085879            (yogurt)
5  0.069572   (root vegetables)
6  0.067767    (tropical fruit)
7  0.060683     (bottled water)
8  0.060349           (sausage)
9  0.053131      (citrus fruit)

Reguł (minlen=2): 10

TOP wg support:
        antecedents       consequents   support  confidence      lift
0        whole milk  other vegetables  0.014837    0.093948  0.769430
1  other vegetables        whole milk  0.014837    0.121511  0.769430
2        rolls/buns        whole milk  0.013968    0.126974  0.804028
3        whole milk        rolls/buns  0.013968    0.088447  0.804028
4              soda        whole milk  0.011629    0.119752  0.758296
5        whole milk     

Oś X — support

Pokazuje jak często dana reguła występuje w całej bazie transakcji.

Support = 0.01 → reguła dotyczy 1% wszystkich transakcji.

Im bardziej w prawo, tym częściej obserwujemy taką kombinację produktów.

Mały support = reguła rzadka (np. egzotyczne połączenia).

Duży support = reguła częsta (produkty kupowane często razem).

To warunkowe prawdopodobieństwo, że jeśli klient kupił A, to kupił też B.

Confidence = 0.13 → w 13% przypadków, gdy kupiono A, kupiono też B.

Im wyżej na wykresie, tym reguła silniejsza logicznie (częściej zachodzi warunek „jeśli A, to B”).

Zwizualizuj wynikowe reguły za pomocą 2 różnych wykresów dynamicznych oraz tabeli
dynamicznej.

In [26]:
fig1 = px.scatter(
    rules_disp,
    x='support', y='confidence',
    color='lift', size='lift',
    hover_data=['antecedents', 'consequents'],
    title='Wykres 1: Reguły – support vs confidence (kolor = lift)'
)
fig1.show()


In [27]:
import networkx as nx
import plotly.graph_objects as go

# graf z reguł (krawędzie: antecedent -> consequent)
G = nx.DiGraph()
for _, row in rules_disp.iterrows():
    G.add_edge(row['antecedents'], row['consequents'],
               weight=row['lift'], support=row['support'], confidence=row['confidence'])

pos = nx.spring_layout(G, seed=42)

edge_x, edge_y = [], []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, line=dict(width=0.8, color='#888'),
    hoverinfo='none', mode='lines')

node_x, node_y, text = [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    text.append(node)

node_trace = go.Scatter(
    x=node_x, y=node_y, mode='markers+text',
    text=text, textposition='top center',
    marker=dict(size=12, color='skyblue'),
    hovertext=text, hoverinfo='text'
)

fig2 = go.Figure(data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Wykres 2: Graf reguł asocjacyjnych (produkty jako węzły)',
        showlegend=False, hovermode='closest',
        margin=dict(b=0,l=0,r=0,t=40))
)
fig2.show()


whole milk jest w centrum – łączy się z wieloma produktami → często kupowany razem z innymi.

yogurt, other vegetables, rolls/buns, soda – produkty kojarzone z mlekiem pełnym w różnych kombinacjach.

Strzałki pokazują kierunek asocjacji: np.
soda → whole milk – jeśli ktoś kupił sodę, często miał też mleko.
whole milk → yogurt – osoby kupujące mleko często dodają jogurt.

In [28]:
import plotly.express as px

# Install Dash and Dash Table
!pip install dash dash-table

# prosty DataTable z filtrowaniem
from dash import dash_table
import pandas as pd
import dash

app = dash.Dash(__name__)
app.layout = dash_table.DataTable(
    data=rules_disp.round(3).to_dict('records'),
    columns=[{"name": i, "id": i} for i in rules_disp.columns],
    filter_action="native",
    sort_action="native",
    page_size=10,
    style_table={'overflowX': 'auto'},
    style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
)

app.run(debug=False, port=8050)

<IPython.core.display.Javascript object>

#punkt d
Znajdź 3 reguły o największym support

In [29]:
top3_support = rules_disp.sort_values('support', ascending=False).head(3)

print("=== 3 reguły o największym support ===")
print(top3_support)


=== 3 reguły o największym support ===
        antecedents       consequents   support  confidence      lift
0        whole milk  other vegetables  0.014837    0.093948  0.769430
1  other vegetables        whole milk  0.014837    0.121511  0.769430
2        rolls/buns        whole milk  0.013968    0.126974  0.804028


Support mówi, jak często dana reguła (kombinacja produktów) pojawia się w całym zbiorze transakcji.

Reguły z najwyższym support to najczęstsze zestawy zakupowe – czyli takie produkty, które klienci najczęściej kupują razem.

W praktyce te 3 reguły to Twój „core” asocjacji – najczęściej współwystępujące pary (lub większe grupy) produktów.

In [30]:
# usuń duplikaty logiczne - ta sama para produktów niezależnie od kierunku
unique_rules = rules_disp.copy()
unique_rules['combo'] = unique_rules.apply(
    lambda r: frozenset(r['antecedents'].split(', ') + r['consequents'].split(', ')), axis=1
)
unique_rules = unique_rules.drop_duplicates(subset='combo')

#pierwsze 3 unikalne reguły o największym support
top3_support = unique_rules.sort_values('support', ascending=False).head(3).copy()

#opis reguły dla etykiety
top3_support['Reguła'] = top3_support['antecedents'] + " ⇒ " + top3_support['consequents']

#wykres słupkowy
fig = px.bar(
    top3_support,
    x='Reguła',
    y='support',
    text='support',
    color='lift',
    color_continuous_scale='viridis',
    title='3 unikalne reguły o największym wsparciu (support)'
)

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(
    xaxis_title='Reguła (A ⇒ B)',
    yaxis_title='Support',
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    width=700,
    height=450
)

fig.show()


#punkt e
Znajdź 3 reguły o największym confidence

In [31]:
#usuń duplikaty logiczne (ta sama para produktów, niezależnie od kierunku) ---
unique_rules = rules_disp.copy()
unique_rules['combo'] = unique_rules.apply(
    lambda r: frozenset(r['antecedents'].split(', ') + r['consequents'].split(', ')), axis=1
)
unique_rules = unique_rules.drop_duplicates(subset='combo')

#wybierz 3 unikalne reguły o najwyższym confidence
top3_confidence = unique_rules.sort_values('confidence', ascending=False).head(3).copy()

#opis reguły
top3_confidence['Reguła'] = top3_confidence['antecedents'] + " ⇒ " + top3_confidence['consequents']

#wykres słupkowy
fig = px.bar(
    top3_confidence,
    x='Reguła',
    y='confidence',
    text='confidence',
    color='lift',
    color_continuous_scale='viridis',
    title='3 unikalne reguły o największej pewności (confidence)'
)

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(
    xaxis_title='Reguła (A ⇒ B)',
    yaxis_title='Confidence',
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    width=700,
    height=450
)

fig.show()


Confidence = P(B∣A)P(B|A)P(B∣A) – prawdopodobieństwo, że klient kupił B, jeśli kupił A.


Im większy confidence, tym bardziej reguła jest wiarygodna.


Na wykresie widzisz 3 najbardziej „pewne” reguły — czyli takie, które mają najwyższe prawdopodobieństwo współwystąpienia.



#punkt f
Znajdź 3 reguły o największym lift

In [32]:

unique_rules = rules_disp.copy()
unique_rules['combo'] = unique_rules.apply(
    lambda r: frozenset(r['antecedents'].split(', ') + r['consequents'].split(', ')), axis=1
)
unique_rules = unique_rules.drop_duplicates(subset='combo')

#wybierz 3 unikalne reguły o najwyższym lift
top3_lift = unique_rules.sort_values('lift', ascending=False).head(3).copy()

#opis reguły dla wykresu
top3_lift['Reguła'] = top3_lift['antecedents'] + " ⇒ " + top3_lift['consequents']

#wykres słupkowy
fig = px.bar(
    top3_lift,
    x='Reguła',
    y='lift',
    text='lift',
    color='support',
    color_continuous_scale='viridis',
    title='3 unikalne reguły o największym współczynniku lift'
)

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(
    xaxis_title='Reguła (A ⇒ B)',
    yaxis_title='Lift',
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    width=700,
    height=450
)

fig.show()


nterpretacja:

Lift > 1 – produkty występują razem częściej niż przypadkowo → silne powiązanie.

Lift = 1 – brak związku, produkty niezależne.

Lift < 1 – produkty pojawiają się razem rzadziej niż oczekiwano → negatywna zależność.

Na wykresie zobaczysz 3 reguły o największym lift, czyli najbardziej „wartościowe” zależności z całego zbioru.

#punkt g
Jeśli w koszyku był produkt whole milk to były tam też … (podaj przynajmniej 3 produkty)

In [33]:
# Filtr: reguły, w których "whole milk" jest po stronie LHS (antecedents)
milk_rules = rules_disp[rules_disp['antecedents'].str.contains(r'\bwhole milk\b', case=False, na=False)]

# Posortuj wg support lub confidence, żeby pokazać najbardziej istotne
milk_rules = milk_rules.sort_values('support', ascending=False)

# Wyświetl top reguły
print("=== Reguły, gdzie w koszyku był 'whole milk' ===")
print(milk_rules.head(10)[['antecedents','consequents','support','confidence','lift']])


=== Reguły, gdzie w koszyku był 'whole milk' ===
  antecedents       consequents   support  confidence      lift
0  whole milk  other vegetables  0.014837    0.093948  0.769430
3  whole milk        rolls/buns  0.013968    0.088447  0.804028
5  whole milk              soda  0.011629    0.073635  0.758296
7  whole milk            yogurt  0.011161    0.070673  0.822940


#punkt h
Do których produktów w koszyku dodawano other vegetables ? (podaj przynajmniej 2
produkty)


In [34]:
# Filtr: reguły, w których "other vegetables" jest po stronie RHS
veg_rules = rules_disp[rules_disp['consequents'].str.contains(r'\bother vegetables\b', case=False, na=False)]

# Posortuj po support lub confidence, żeby mieć najistotniejsze
veg_rules = veg_rules.sort_values('support', ascending=False)

# Wyświetl najważniejsze reguły
print("=== Reguły, gdzie do koszyka dodawano 'other vegetables' ===")
print(veg_rules.head(10)[['antecedents','consequents','support','confidence','lift']])


=== Reguły, gdzie do koszyka dodawano 'other vegetables' ===
  antecedents       consequents   support  confidence      lift
0  whole milk  other vegetables  0.014837    0.093948  0.769430
8  rolls/buns  other vegetables  0.010559    0.095990  0.786154


In [35]:
top2_veg = veg_rules.head(2)
print("\n2 produkty, do których najczęściej dodawano 'other vegetables':")
for item in top2_veg['antecedents']:
    print("-", item)



2 produkty, do których najczęściej dodawano 'other vegetables':
- whole milk
- rolls/buns
